# Overview

This notebook aims to implement optimal power flow (OPF) using second order cone programming (SOCP) using the bus-injection formulation. The formulations used are taken from [1]. 

In this formulation we take care the handle reactive power $Q$ and real power $P$ seperately instead of as a single complex value $S$, which most solvers can't handle

#### Sources
[1] Convex Relaxations of Optimal Power Flow, Part I. By Steven Low

# Formulation

Consider $(n+1)\times (n+1)$ admittance matrix
$$
Y_{ij} = \begin{cases}\sum_{k:(k, i)\in E}y_{ik} & i = j \\
-y_{ij} & i\neq j \text{ and } (i, j)\in E \\
0 & \text{otherwise}
    \end{cases}
    $$

where $y_{ij} = 1/z_{ij}$ (defined in [1].II.B) is the admittance (one-over-impedance) if that node. Note that when considering matrices $Y \neq Z^{-1}$. $Y$ is $n+1$ to take the source bus into account (I assume). So at the diagonal entries we are considering the admittance between a node and itself (weird), and we use as admittance the sum of all connecting nodes. At off-diagonal elements which do have connectiongs we take $-y_{ij}$.

Now introduce (n+1)-dimensional vector $e_j$:
$$
e_j = \begin{cases}1 & \text{ at index j }\\

0 & \text{otherwise}
    \end{cases}
    $$

using $e_j$ we define $$Y_j := e_j e_j^H Y$$

This is an $(n+1)\times (n+1)$ matrix, where the jth is equal to the jth row of $Y$, and other rows are the zero vector:

We can split up each $Y_j$ in it's real and imaginary parts:
$$\Phi_j := \frac{1}{2}(Y_j^H + Y_j)$$
$$\Psi_j := \frac{1}{2i}(Y_j^H - Y_j)$$

These values are useful since we can use them to get $p_j$ and $q_j$:
$$p_j = \text{Re}[s_j] = V^H\Phi_jV \tag{a}$$
$$q_j = \text{Im}[s_j] = V^H\Psi_j \tag{b}V$$

Lastly we introduce
 $$J := e_je_j^H$$ 
 which yields a matrix with a single $1$ on entry $(j, j)$ and zero everywhere else.
 This can than be used to state 
$V^H J_j V = v_j$.

## Generalized SOCP formulation

From [1] we get eq. (23) which states that OPF-socp is:
$$\min_{W_G}C(W_G) \text{ subject to } W_G\in\mathbb{W}_G^+$$

where 
$$
[W_G]_{ij} = \begin{cases}
|V_j|^2 & \text{i=j} \\
V_jV_k^H & \text{j != i}
    \end{cases}
    $$

$$\mathbb{W}_G^+ := \{W_G|W_G \text{ satisfies } (12),W_G{j,k}\succeq\, (j,k)\in\mathcal{E}\}$$

and (12) mentioned in the definition of $\mathbb{W}_G^+$ is
$$\underline{s}_j \leq \sum_{k:(j,k)\in E} y_{jk}^H([W_G]_{jj} - [W_G]_{jk}) \leq \bar{s}_j\tag{12a}$$
$$\underline{v}_j \leq [W_G]_{jj} \leq \bar{v}_j \tag{12b}$$

To avoid working with complex numbers we need to rewrite (12a) in the terms of (a) and (b). This is shown at the top of page 13 in [1]:
$$\underline{p}_j \leq \text{tr}[\Phi_jW_G]\ \leq \overline{p}_j  \tag{\text{con. }1} $$
$$\underline{q}_j \leq \text{tr}[\Psi_jW_G]\ \leq \overline{q}_j \tag{\text{con. }2}$$

$$\underline{v}_j \leq \text{tr}[J_jW_G]\ \leq \overline{v}_j \tag{\text{con. }2}$$
where $\text{tr}[X]$ is the trace operator, which takes the sum over all elements in the diagonal of $X$.

## Computing $\Psi_j$ and $\Phi_j$
Assume you have the following admittance matrix:

$$ Y = \begin{bmatrix}
Y_{AA} & Y_{AB} & Y_{AC} & Y_{AD} \\
Y_{AB} & Y_{BB} & Y_{BC} & Y_{BD} \\
Y_{AC} & Y_{BC} & Y_{CC} & Y_{CD} \\
Y_{AD} & Y_{BD} & Y_{CD} & Y_{DD} 
\end{bmatrix}  $$
here we have 
$$ Y_1 = \begin{bmatrix}
0 & 0 & 0 & 0 \\
Y_{AB} & Y_{BB} & Y_{BC} & Y_{BD} \\
0 & 0 & 0 & 0 \\
0 & 0 & 0 & 0
\end{bmatrix}  $$
and 
$$ Y_1^H = \begin{bmatrix}
0 & Y_{AB}* & 0  & 0  \\
0 & Y_{BB}* & 0  & 0  \\
0 & Y_{BC}* & 0  & 0  \\
0 & Y_{BD}* & 0  & 0  
\end{bmatrix}  $$

which leads to
$$ \Phi_1 = \frac{1}{2}\Big(\begin{bmatrix}
0 & Y_{AB}* & 0  & 0  \\
0 & Y_{BB}* & 0  & 0  \\
0 & Y_{BC}* & 0  & 0  \\
0 & Y_{BD}* & 0  & 0  
\end{bmatrix} + \begin{bmatrix}
0 & 0 & 0 & 0 \\
Y_{AB} & Y_{BB} & Y_{BC} & Y_{BD} \\
0 & 0 & 0 & 0 \\
0 & 0 & 0 & 0
\end{bmatrix}\Big) = \frac{1}{2}\begin{bmatrix}
0 & Y_{AB}* & 0 & 0 \\
Y_{AB} & 2\mathbb{Re}[Y_{BB}] & Y_{BC} & Y_{BD} \\
0 & Y_{BC}* & 0 & 0 \\
0 & Y_{BC}* & 0 & 0
\end{bmatrix}   $$

note the $BB$ entry, which holds since
$$Y* + Y = (Y_r - jY_i) + (Y_r + jY_i) = 2Y_r \tag{H1}$$

Now for $\Psi_j$, we can compute

$$ \Psi_1 = \frac{1}{j2}\Big(\begin{bmatrix}
0 & Y_{AB}* & 0  & 0  \\
0 & Y_{BB}* & 0  & 0  \\
0 & Y_{BC}* & 0  & 0  \\
0 & Y_{BD}* & 0  & 0  
\end{bmatrix} - \begin{bmatrix}
0 & 0 & 0 & 0 \\
Y_{AB} & Y_{BB} & Y_{BC} & Y_{BD} \\
0 & 0 & 0 & 0 \\
0 & 0 & 0 & 0
\end{bmatrix}\Big) = \frac{1}{j2}\begin{bmatrix}
0 & -Y_{AB}* & 0 & 0 \\
Y_{AB} & -2j\mathbb{Im}[Y_{BB}] & Y_{BC} & Y_{BD} \\
0 & -Y_{BC}* & 0 & 0 \\
0 & -Y_{BC}* & 0 & 0
\end{bmatrix}   $$

again note the $BB$ entry, which holds since
$$Y* - Y = (Y_r - jY_i) - (Y_r = jY_i) = -2jY_i \tag{H2}$$

From this we can conclude that:
- For $\Psi_j$ and $\Phi_j$ the diagonals are real (the $j$ in the diagonal of $\Psi_1$ is cancelled by the $\frac{1}{2j}$ in front of it)
- For $\Phi_j$ we get that opposing elements ($Y_AB \xleftrightarrow{} Y_BA$) are each others conjugate
- For $\Psi_j$ we get that opposing elements ($Y_AB \xleftrightarrow{} Y_BA$) are each others conjugate and NEGATIVE

## Computing $\text{tr}[\Phi_jW_G]$
Assume we have
$$ \Phi_1 = \frac{1}{2}\begin{bmatrix}
0 & Y_{AB}* & 0 \\
Y_{AB} & 2\mathbb{Re}[Y_{BB}] & Y_{BC} \\
0 & Y_{BC}* & 0 
\end{bmatrix}   $$

$$
W_G = \begin{bmatrix}
W_{AA} & W_{AB} & W_{AC} \\
W_{BA} & W_{BB} & W_{BC} \\
W_{CA} & W_{CB} & W_{CC} \\
\end{bmatrix} = \begin{bmatrix}
|V_A|^2 & V_AV_B^* & V_AV_C^* \\
V_B V_A^* & |V_B|^2 & V_BV_C^*\\
V_C V_A^* & V_C V_B^* & |V_C|^2 \\
\end{bmatrix} $$

Now compute $p_j = \text{tr}[\Phi_jW_G]$:

$$
p_j = \text{tr}\Big[\begin{bmatrix}
0 & \frac{1}{2}Y_{AB}* & 0 \\
\frac{1}{2}Y_{AB} & \mathbb{Re}[Y_{BB}] & \frac{1}{2}Y_{BC} \\
0 & \frac{1}{2}Y_{BC}* & 0 
\end{bmatrix} \begin{bmatrix}
|V_A|^2 & V_AV_B^* & V_AV_C^* \\
V_B V_A^* & |V_B|^2 & V_BV_C^*\\
V_C V_A^* & V_C V_B^* & |V_C|^2 \\
\end{bmatrix}\Big] $$
we only consider the diagonal calculations, since we'll later take the trace operator anyway:
so
$$[\Phi_j W_G]_{00} = 0\cdot |V_A|^2+\frac{1}{2}Y_{AB}^*V_B V_A^* + 0\cdot V_C V_A^* = \frac{1}{2}Y_{AB}^*V_BV_A^* $$
$$[\Phi_j W_G]_{11} = \frac{1}{2}Y_{AB}V_A V_B^* + \mathbb{Re}[Y_{BB}]|V_B|^2 + \frac{1}{2}Y_{BC}V_CV_B^* $$
$$[\Phi_j W_G]_{22} = 0\cdot V_AV_C^* + \frac{1}{2}Y_{BC}^* V_B V_C^* + 0\cdot |V_C|^2 = \frac{1}{2}Y_{BC}^* V_B V_C^* $$
$$\text{tr}[\Phi_j W_G] = \Phi_j W_G]_{00} + \Phi_j W_G]_{11} + \Phi_j W_G]_{22}$$
$$\text{tr}[\Phi_j W_G] = \frac{1}{2}Y_{AB}^*V_BV_A^* + \frac{1}{2}Y_{AB}V_A V_B^* + \mathbb{Re}[Y_{BB}]|V_B|^2 + \frac{1}{2}Y_{BC}V_CV_B^* + \frac{1}{2}Y_{BC}^* V_B V_C^*$$
note that we're again adding two conjugates together (similar as eq. H1), only now of composed terms: $Y_{AB}^*V_BV_A^*$ is the conjugate of $Y_{AB}V_A V_B^*$. So if we compute $Y_{AB}^*V_BV_A^*$, we know their sum is $2\Re{Y_{AB}^*V_BV_A^*}$. The same can be done for $\frac{1}{2}Y_{BC}V_CV_B^* + \frac{1}{2}Y_{BC}^* V_B V_C^*$.  


## Computing $\text{tr}[\Psi_jW_G]$
Assume we have
$$ \Psi_1 = \frac{1}{2j}\begin{bmatrix}
0 & -Y_{AB}* & 0 \\
Y_{AB} & -2j\mathbb{Im}[Y_{BB}] & Y_{BC} \\
0 & -Y_{BC}* & 0 
\end{bmatrix} = \begin{bmatrix}
0 & -\frac{1}{2j}Y_{AB}* & 0 \\
\frac{1}{2j}Y_{AB} & -\mathbb{Im}[Y_{BB}] & \frac{1}{2j}Y_{BC} \\
0 & -\frac{1}{2j}Y_{BC}* & 0 
\end{bmatrix} $$
and 
$$
W_G = \begin{bmatrix}
W_{AA} & W_{AB} & W_{AC} \\
W_{BA} & W_{BB} & W_{BC} \\
W_{CA} & W_{CB} & W_{CC} \\
\end{bmatrix} = \begin{bmatrix}
|V_A|^2 & V_AV_B^* & V_AV_C^* \\
V_B V_A^* & |V_B|^2 & V_BV_C^*\\
V_C V_A^* & V_C V_B^* & |V_C|^2 \\
\end{bmatrix} $$

Now compute $q_j = \text{tr}[\Psi_jW_G]$:
$$[\Psi_jW_G]_{00} = 0\cdot |V_A|^2 + \frac{-1}{2j}Y_{AB}^*V_BV_A^* + 0\cdot V_CV_A^* = \frac{-1}{2j}Y_{AB}^*V_BV_A^* $$
$$[\Psi_jW_G]_{11} = \frac{1}{2j}Y_{AB} V_A V_B^* + -\mathbb{Im}[Y_BB]|V_B|^2 + \frac{1}{2j}Y_{BC}V_C V_B^*$$
$$[\Psi_jW_G]_{22} = 0 \cdot V_A V_C^* + -\frac{1}{2j}Y_{BC}^*V_BV_C^* + 0\cdot |V_C|^* = -\frac{1}{2j}Y_{BC}^*V_BV_C^*$$

Adding these together (as you'd do in the trace operator) we get:
$$\text{tr}[\Psi_j W_G] = [\Psi_jW_G]_{00} + [\Psi_j W_G]_{11} + [\Psi_j W_G]_{22}$$
$$\text{tr}[\Psi_j W_G] = \frac{-1}{2j}Y_{AB}^*V_BV_A^* + \frac{1}{2j}Y_{AB} V_A V_B^* + - \mathbb{Im} [Y_{BB}]|V_B|^2 + \frac{1}{2j}Y_{BC}V_C V_B^* +  -\frac{1}{2j}Y_{BC}^*V_BV_C^*$$

Note that in this equation, we're subtracting conjugates, as in eq. (H2). To illustrate, lets define 
$$Y_{AB} V_A V_B^* := M$$
In this case we see

$\frac{1}{2j}(M - M^*) = \frac{1}{2j}(2j \mathbb{Im}[M]) = \mathbb{Im}[M]$. 

This can be done for $\frac{-1}{2j}Y_{AB}^*V_BV_A^* + \frac{1}{2j}Y_{AB} V_A V_B^*$ and $\frac{1}{2j}Y_{BC}V_C V_B^* +  -\frac{1}{2j}Y_{BC}^*V_BV_C^*$. This means that all elements in the diagonal are only dependent on the imaginary parts. Since taking the imaginary part of a complex number results in a real value, this means the sum is a real value.

## Congestion objective
The objective should meet the following three criteria:
- When a cable does not exceed the limit, it should not be considered congested
- Congestion can occur both ways
- Exceeding the cable rating a lot is much worse than exceeding it a little.

This all leads to the following objective:
$$\text{objective: } \min_W \sum_{(j,k)\in E}\big(\text{min}(0, |I_{jk}(W)| - I_{jk,\text{max}})\big)^2 \tag{\text{obj. }1}$$

Or similarly, defined using power:

$$\text{objective: } \min_W \sum_{(j,k)\in E}\big(\text{min}(0, |P_{jk}(W)| - P_{jk,\text{max}})\big)^2 \tag{\text{obj. }2}$$$$


## Calculating currents from node voltages
To calculate the current from node voltages, we need to do he following. First recall that 
$$I_{mn} = \frac{V_m-V_n}{Z_{mn}}$$ 

And realize we only care about the magnitude of the current:

$$|I_{mn}| = |V_{mn}Y_{mn}|$$
$$|I_{mn}| = |V_{mn}||Y_{mn}|$$
$$|I_{mn}| = |V_m - V_n||Y_{mn}|$$
$$|I_{mn}|^2 = |V_m - V_n|^2|Y_{mn}|^2$$
$$|I_{mn}|^2 = \big((\mathbb{Re}[V_m] - \mathbb{Re}[V_n])^2 + (\mathbb{Im}[V_m] - \mathbb{Im}[V_n])^2\big)|Y_{mn}|^2$$
$$|I_{mn}|^2 = \big(|V_m|^2 + |V_n|^2 -2(\mathbb{Re}[V_m]\mathbb{Re}[V_n] + \mathbb{Im}[V_m]\mathbb{Im}[V_n])\big)|Y_{mn}|^2$$

note that $2(\mathbb{Re}[V_m]\mathbb{Re}[V_n] + \mathbb{Im}[V_m]\mathbb{Im}[V_n]) = W_{mn} + W_{nm}$
$$|I_{mn}|^2 = \big(|V_m|^2 + |V_n|^2 -(W_{mn} + W_{nm})\big)|Y_{mn}|^2$$
$$|I_{mn}|^2 = \big(W_{mm} + W_{nn} - W_{mn} - W_{nm}\big)|Y_{mn}|^2 \tag{9}$$

## Calculating powers from line currents
Recall that 

$$P = VI = (I\cdot R)I = I^2\cdot R = \frac{I^2}{G} = \frac{I^2}{\Re[Y]}  \tag{10}$$

By combining (9), (10) we get:
$$P_{mn} = \frac{I_{mn}^2}{G_{mn}}$$
$$P_{mn} = \frac{\big(W_{mm} + W_{nn} - W_{mn} - W_{nm}\big)|Y_{mn}|^2}{G_{mn}}$$
$$P_{mn} = \big(W_{mm} + W_{nn} - W_{mn} - W_{nm}\big)\mathcal{A}_{mn} \tag{11}$$
where 
$$\mathcal{A}_{mn} := \frac{|Y_{mn}|^2}{G_{mn}}$$

Now, consider that lines are normally rated by a maximum current (see eq. (obj. 1)). However, this can easily be converted to the rated power using:
$$P_{mn,\text{max}} = I_{mn,\text{max}}^2R_{mn} $$

Combining (11) and (obj. 2) we get:
$$\sum_{(j,k)\in E}\big(\text{min}(0, |\big(W_{mm} + W_{nn} - W_{mn} - W_{nm}\big)|Y_{mn}|| - P_{mn,\text{max}})\big)^2 \tag{\text{obj. }3}$$



## OPTIONAL: Power factor
In Dutch distribution grids, the PF needs to be above 0.85 (https://nl.wikipedia.org/wiki/Cos_%CF%86-compensatie). Power Factor is defined as
$$PF := \frac{P}{S} = \frac{P}{\sqrt{P^2 + Q^2}}$$

So we'd need

$$ \frac{P}{\sqrt{P^2 + Q^2}} \geq \text{PF}$$

this can be rewritten to
$$ P \geq \text{PF}\cdot \sqrt{P^2 + Q^2}$$
$$ P^2 \geq \text{PF}^2\cdot (P^2 + Q^2)$$
$$ (1- \text{PF}^2)P^2 - \text{PF}^2Q^2 \geq 0$$
$$ (\text{PF}^2-1)P^2 + \text{PF}^2Q^2 \leq 0 $$
$$ \sqrt{(\text{PF}^2-1)P^2 + \text{PF}^2Q^2} \leq 0 $$
Note that
$$ \sqrt{(\text{PF}^2-1)P^2 + \text{PF}^2Q^2} = \sqrt{\big(\sqrt{\text{PF}^2 - 1}\cdot P\big)^2 + (\text{PF}\cdot Q)^2 } = \begin{bmatrix}\sqrt{\text{PF}^2 - 1}\cdot P \\  \text{PF}\cdot Q \end{bmatrix}_2$$

which means that
$$\begin{bmatrix}\sqrt{\text{PF}^2 - 1}P \\  \text{PF}\cdot Q \end{bmatrix}_2 \leq 0$$






## Constraints summarized
Whenever real/imag is used, these elements are already fully real/imag, but their cvxpy datatype is still complex. Complex datatypes don't allow for inqualities, so we have to set these matrices to strictly real/imag
$$W \succeq 0$$
$$\underline{v_j} \leq \text{tr}\big[\mathbb{Re}[J_jW]\big] \leq \overline{v_j} \hspace{1em} \forall \hspace{1em} j \in {0, 1, ..., N}$$
$$\underline{p_j} \leq \text{tr}\big[\mathbb{Re}[\Phi_jW]\big] \leq \overline{q_j} \hspace{1em} \forall \hspace{1em} j \in {0, 1, ..., N}$$
$$\underline{q_j} \leq \text{tr}\big[\mathbb{Im}[\Psi_jW]\big] \leq \overline{q_j} \hspace{1em} \forall \hspace{1em} j \in {0, 1, ..., N}$$


## Extracting answers
The tutorial at the top stops the OPF definition here. This means they define the basic power flow equations and voltage limits, using only $W_G$, from which the voltages at all nodes can be derived as follows. Let $\mathbb{P}_j$ denote the unique path from node 0 to arbitrary node $j$. For $j = 1, ..., n$
$$|V_j| := \sqrt{[W_G]_{jj}}$$
$$\angle V_J := -\sum_{(i,k)\in\mathbb{P}_j}\angle [W_G]_{ik}$$

In case we're interested in the powers at a node instead of the voltages, we can use

$$p_j = \text{tr}\big[\mathbb{Re}[\Phi_jW_G]\big]\$$
$$q_j = \text{tr}\big[\mathbb{Im}[\Psi_jW_G]\big]\$$


## Pre processing

In [1]:
import numpy as np
import cvxpy as cp
import utils.network_utilities as nu


In [2]:
network = nu.make_simple_mesh_net()
n_nodes = len(network["node"]["id"])

In [3]:
Y = nu.make_admittance_matrix(network)
ejs = [nu.make_ej(n_nodes, j) for j in range(n_nodes)]
Jjs = [nu.make_Jj(Y, j) for j in range(n_nodes)]
Yjs = [nu.make_Yj(Y, j) for j in range(n_nodes)]
Psijs = [nu.make_Psij(Y, j) for j in range(n_nodes)]
Phijs = [nu.make_Phij(Y, j) for j in range(n_nodes)]

## OPF

In [4]:
n_nodes = len(network["node"]["id"])
W = cp.Variable((n_nodes, n_nodes), complex=True)

ADD_LINE_CONSTRAINTS = False
ADD_POWER_FACTOR_CONSTRAINTS = False
OBJECTIVE = "congestion"

In [5]:
"""
SDF Relaxation
"""
constraints = [W >> 0]


"""
voltage limit constraints. 
We need to take the real part of 'Jjs[j]@W' since currently W is a complex variable, and we can't have inequalities with complex variables.
However, the diagional of Jjs[j]@W (which is the only part we use in trace operator) turns out to only have a real part if you compute it
"""
constraints += [network["node"]["v_min"][j] <= cp.trace(cp.real(Jjs[j]@W)) for j in range(n_nodes)]
constraints += [network["node"]["v_max"][j] >= cp.trace(cp.real(Jjs[j]@W)) for j in range(n_nodes)]

"""
Real power constraints. 
We need to take the real part of 'Phijs[j]@W' since Phijs[j] and W are complex variables, and we can't have inequalities with complex variables.
However, the diagional of 'Phijs[j]@W' (which is the only part we use in trace operator) turns out to only have a real part if you compute it
so this formulation is valid.
"""
constraints += [network["node"]["p_min"][j] <= cp.trace(cp.real(Phijs[j]@W)) for j in range(n_nodes)]
constraints += [network["node"]["p_max"][j] >= cp.trace(cp.real(Phijs[j]@W)) for j in range(n_nodes)]

"""
Reactive power constraints. 
We need to take the imag part of 'Psijs[j]@W' since Psijs[j] and W are complex variables, and we can't have inequalities with complex variables.
However, the diagional of 'Psijs[j]@W' (which is the only part we use in trace operator) turns out to only have a imag part if you compute it 
so this formulation is valid.
"""
constraints += [network["node"]["q_min"][j] <= cp.trace(cp.imag(Psijs[j]@W)) for j in range(n_nodes)]
constraints += [network["node"]["q_max"][j] >= cp.trace(cp.imag(Psijs[j]@W)) for j in range(n_nodes)]

if ADD_POWER_FACTOR_CONSTRAINTS:
    for j in range(n_nodes):

        q = cp.trace(cp.imag(Psijs[j]@W))  # see eq. (con1)
        p = cp.trace(cp.real(Phijs[j]@W))  # see eq. (con1)

        
        PF = 0.85 # power factor
        c_q = PF  # constant prefix in front of reactive power term term in norm (see section "optional: POWER FACTOR")
        c_r = np.sqrt(1-PF**2) # constant prefix in front of real power term in norm (see section "optional: POWER FACTOR")

        constraints += [cp.SOC(0, cp.vstack([c_r*p,c_q*q]))]

"""
Line current constraints.
Although not defined in [1], I think it is valuable to add line constraints. Not sure if this formulation is correct though. I think taking the square accounts for both 
"""
if ADD_LINE_CONSTRAINTS:
    for line_idx, (from_node_id, to_node_id) in enumerate(zip(network["line"]["from_node"], network["line"]["to_node"])):
        to_node_idx, from_node_idx = nu.id_to_index(network["node"], to_node_id), nu.id_to_index(network["node"], from_node_id)
        line_constraint = network["line"]["i_max"][line_idx]**2/(cp.norm(network["line"]["y1"][line_idx])**2) >= cp.real(W[from_node_idx, from_node_idx]) + cp.real(W[to_node_idx, to_node_idx]) + cp.real(W[to_node_idx, from_node_idx] + W[from_node_idx, to_node_idx])
        constraints += [line_constraint]




$$\sum_{(j,k)\in E}\big(\text{min}(0, |\big(W_{mm} + W_{nn} - W_{mn} - W_{nm}\big)|Y_{mn}|| - P_{mn,\text{max}})\big)^2 \tag{\text{obj. }3}$$



In [14]:
if OBJECTIVE == "mock":
    objective = cp.Minimize(cp.trace(cp.real(W)))
elif OBJECTIVE == "congestion":
    obj = 0

    # obj += cp.maximum(0, cp.abs(W[0, 0]))
    # obj += cp.maximum(0, cp.abs(W[1, 1]))
    # objective = cp.Minimize(obj)

    for line_idx, (from_node_id, to_node_id) in enumerate(zip(network["line"]["from_node"], network["line"]["to_node"])):
        i1 = nu.id_to_index(network["node"], from_node_id)
        i2 = nu.id_to_index(network["node"], to_node_id)
        y = network["line"]["y1"][line_idx]
        r = network["line"]["r1"][line_idx]
        
        A_mn = cp.abs(y)**2 * r # cp.real(y)
        obj += cp.maximum(0, cp.abs(W[i1, i1] + W[i2, i2] - W[i1, i2] - W[i2, i1])*A_mn)


    objective = cp.Minimize(obj)

else:
    raise Exception

In [15]:
# mock objective; just to test of the constraints are valid


In [16]:
problem = cp.Problem(objective, constraints) 

In [17]:
problem.solve()

4.283942741348027e-08

In [ ]:
network["line"]

{'id': [3, 5, 8],
 'from_node': [1, 2, 1],
 'to_node': [2, 6, 6],
 'r1': [0.25, 0.25, 0.25],
 'x1': [0.2, 0.2, 0.2],
 'i_max': [1000, 1000, 1000],
 'incoming_line_indices': [[], [0], []],
 'outgoing_line_indices': [[1], [], []],
 'p_max': [250000.0, 250000.0, 250000.0],
 'z1': [(0.25+0.2j), (0.25+0.2j), (0.25+0.2j)],
 'y1': [(2.4390243902439024-1.951219512195122j),
  (2.4390243902439024-1.951219512195122j),
  (2.4390243902439024-1.951219512195122j)]}

ModuleNotFoundError: No module named 'matplotlib'